# My own solution after studying the data set and other kaggle notebooks

In [3]:
# Handling the general imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

In [4]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

combine = [train_df, test_df]

## Inspecting & Preprocessing the Data

In [28]:
# Inspecting the data
print(train_df.info())
print('-' * 100)
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count 

In [32]:
# Checking for missing values
print(train_df.isnull().sum() / len(train_df) * 100)
print('-' * 100)
print(test_df.isnull().sum() / len(test_df) * 100)

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.000000
dtype: float64
----------------------------------------------------------------------------------------------------
PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          78.229665
Embarked        0.000000
dtype: float64


Since for the 'Cabin' feature most of its values are missing (over 75% in both the train and test data), we decide to not use this feature and therefore do not need to fill these missing values. However, we will fill the missing values for 'Age', 'Embarked', and 'Fare'.

In [33]:
# Fill missing values
for df in combine:
    df['Age'].fillna(df['Age'].median(), inplace = True) # fill missing age values with median (we could also try the rounded mean in this case)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True) # fill missing embarked values with the mode
    df['Fare'].fillna(df['Fare'].median(), inplace = True) # fill missing fare values with the median

In [34]:
# Checking for missing values
print(train_df.isnull().sum() / len(train_df) * 100)
print('-' * 100)
print(test_df.isnull().sum() / len(test_df) * 100)

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.000000
dtype: float64
----------------------------------------------------------------------------------------------------
PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          78.229665
Embarked        0.000000
dtype: float64


Now, we can add some features which we might find useful and later on check if they improve our results when trying different machine learning algorithms.

In [40]:
for df in combine:
    df['FamilySize'] = df['SibSp'] + df['Parch']
    
    df['isAlone'] = 0
    df['isAlone'].loc[df['FamilySize'] > 0] = 1

/var/folders/fx/4xk4qzyx6s184pwwl260p0gr0000gn/T/ipykernel_2993/3585168046.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isAlone'].loc[df['FamilySize'] > 0] = 1


In [41]:
train_df.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,isAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q,0,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,4,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,2,1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,1,1


We can see that not all the features we want to use consist of numerical values. This could be a problem, so let us convert those into nuemrical values.

In [44]:
for df in combine:
    df['Sex'].replace({'male' : 0, 'female' : 1}, inplace = True)
    df['Embarked'].replace({'S' : 0, 'C' : 1, 'Q' : 2}, inplace = True)

In [45]:
train_df.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,isAlone
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,1,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,1,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,0,0
5,6,0,3,"Moran, Mr. James",0,28.0,0,0,330877,8.4583,NaN,2,0,0
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,E46,0,0,0
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.0750,NaN,0,4,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,NaN,0,2,1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NaN,1,1,1


### Inspecting the correlation

In [53]:
for feat in ['Pclass', 'SibSp', 'Parch', 'Embarked', 'FamilySize', 'isAlone']: 
    print('Survival correlation by {}'.format(feat))
    print(train_df[[feat, 'Survived']].groupby(feat, as_index = False).mean().sort_values(by = 'Survived', ascending = False))


Survival correlation by Pclass
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
Survival correlation by SibSp
   SibSp  Survived
1      1  0.535885
2      2  0.464286
0      0  0.345395
3      3  0.250000
4      4  0.166667
5      5  0.000000
6      8  0.000000
Survival correlation by Parch
   Parch  Survived
3      3  0.600000
1      1  0.550847
2      2  0.500000
0      0  0.343658
5      5  0.200000
4      4  0.000000
6      6  0.000000
Survival correlation by Embarked
   Embarked  Survived
1         1  0.553571
2         2  0.389610
0         0  0.339009
Survival correlation by FamilySize
   FamilySize  Survived
3           3  0.724138
2           2  0.578431
1           1  0.552795
6           6  0.333333
0           0  0.303538
4           4  0.200000
5           5  0.136364
7           7  0.000000
8          10  0.000000
Survival correlation by isAlone
   isAlone  Survived
1        1  0.505650
0        0  0.303538
